In [1]:
import csv
import pandas as pd
import pathlib

In [2]:
path = '/home/xandao/Documentos/resultados_gimp/u-net'

In [8]:
list_files = [file for file in pathlib.Path(path).rglob('mean.csv') if file.is_file()]
len(list_files)

6

In [4]:
index = ['loss', 'dice', 'jaccard', 'precision', 'recall']#, 'folder', 'best_fold']
index = [i + '_' + mean_std for i in index for mean_std in ['mean', 'std']]
sheet_rgb = pd.DataFrame(index)
sheet_grayscale = pd.DataFrame(index)
display(sheet_rgb)

,0
0,loss_mean
1,loss_std
2,dice_mean
3,dice_std
4,jaccard_mean
5,jaccard_std
6,precision_mean
7,precision_std
8,recall_mean
9,recall_std


In [11]:
def round_mean(value):
     return '=ROUND(' + str(value) + '; 4)'

def plus_minus_std(value):
     return '="±"&ROUND(' + str(value) + '; 4)'

def get_mean(sheet, metric):
    if metric not in ['train', 'val', 'test']:
        raise ValueError('metric not valid')

    mean_metric = 'mean_' + metric
    std_metric = 'std_' + metric
    return [
        round_mean(sheet.loc['loss'][mean_metric]), plus_minus_std(sheet.loc['loss'][std_metric]),
        round_mean(sheet.loc['dice'][mean_metric]), plus_minus_std(sheet.loc['dice'][std_metric]),
        round_mean(sheet.loc['jaccard'][mean_metric]), plus_minus_std(sheet.loc['jaccard'][std_metric]),
        round_mean(sheet.loc['precision'][mean_metric]), plus_minus_std(sheet.loc['precision'][std_metric]),
        round_mean(sheet.loc['recall'][mean_metric]), plus_minus_std(sheet.loc['recall'][std_metric]),
    ]


def insert_mean_sheet(sheet, mean_train, mean_val, mean_test):
    sheet['mean_train_' + str(image_size)] = mean_train
    sheet['mean_val_' + str(image_size)] = mean_val
    sheet['mean_test_' + str(image_size)] = mean_test

def save_xlsx_csv(filename, sheet):
    sheet.to_excel(filename + '.xlsx', na_rep='', engine='xlsxwriter')
    sheet.to_csv(filename + '.csv', sep=';', na_rep='', quoting=csv.QUOTE_ALL, index=None)

for file in sorted(list_files):
    sheet_cfg = pd.read_csv(str(file).replace('mean.csv', 'cfg.csv'), header=None, sep=';', index_col=0)
    color_mode = sheet_cfg.loc['channel'][1]
    image_size = sheet_cfg.loc['image_size'][1]

    sheet_mean = pd.read_csv(file, sep=';', index_col=0)
    mean_train = get_mean(sheet_mean, 'train')
    mean_val = get_mean(sheet_mean, 'val')
    mean_test = get_mean(sheet_mean, 'test')

    if color_mode == str(1):
        insert_mean_sheet(sheet_grayscale, mean_train, mean_val, mean_test)
    else:
        insert_mean_sheet(sheet_rgb, mean_train, mean_val, mean_test)
save_xlsx_csv('mean_grayscale', sheet_grayscale)
save_xlsx_csv('mean_rgb', sheet_rgb)